In [1]:
import pandas as pd
import numpy as np 
from os import path


In [3]:
DATA_DIR = "../../data"

In [5]:
chen_train = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train_data.csv"), index_col=0)
chen_valid = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_valid_data.csv"), index_col=0)
chen_train.head()

,Antibody_ID,heavy,light,Y
2073,6aod,EVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLE...,DIVMTKSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKR...,0
1517,4yny,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,EFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2025,5xcv,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,QFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2070,6and,EVQLVESGGGLVQPGGSLRLSCAASGYEFSRSWMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRSSQSIVHSVGNTFLEWYQQKPG...,1
666,2xqy,QVQLQQPGAELVKPGASVKMSCKASGYSFTSYWMNWVKQRPGRGLE...,DIVLTQSPASLALSLGQRATISCRASKSVSTSGYSYMYWYQQKPGQ...,0


In [6]:
with open(path.join(DATA_DIR, "fairseq/train.input0"), "w") as dat:
    with open(path.join(DATA_DIR, "fairseq/train.label"), "w") as lab:
        for i, row in chen_train.iterrows():
            dat.write(row["heavy"] + row["light"] + '\n')
            lab.write(str(row["Y"]) + '\n')

In [9]:
train_seqs = [row["heavy"] + row["light"] for i, row in chen_train.iterrows()]

In [7]:
with open(path.join(DATA_DIR, "fairseq/dev.input0"), "w") as dat:
    with open(path.join(DATA_DIR, "fairseq/dev.label"), "w") as lab:
        for i, row in chen_valid.iterrows():
            dat.write(row["heavy"] + row["light"] + '\n')
            lab.write(str(row["Y"]) + '\n')

In [10]:
valid_seqs = [row["heavy"] + row["light"] for i, row in chen_valid.iterrows()]

In [8]:
from fairseq.data import Dictionary
from collections import Counter

In [11]:
token_counter = Counter()
for record in train_seqs:
    token_counter.update(record)
    
dictionary = Dictionary()
for token, count in sorted(token_counter.items()):
    dictionary.add_symbol(token, count)
    
with open(path.join(DATA_DIR, "fairseq/dict.txt"), "w") as f:
    dictionary.save(f)

In [60]:
!fairseq-train ../../data/fairseq/ \
    --restore-file ../../../../BioPhi/biophi/humanization/methods/sapiens/models/v1/checkpoint_vh.pt \
    --reset-optimizer --reset-dataloader --reset-meters \
    --dataset-impl raw \
    --task sentence_prediction \
    --criterion sentence_prediction \
    --user-dir ../../../../BioPhi/biophi/humanization/methods/sapiens \
    --arch roberta_small \
    --max-tokens 8096 \
    --update-freq 4 \
    --lr 1e-4 \
    --optimizer adam \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 16000 \
    --max-positions 512 \
    --skip-invalid-size-inputs-valid-test \
    --validate-interval-updates 5000 \
    --save-interval-updates 5000 \
    --save-dir ../../data/models/fairseq \
    --num-classes 2

2022-01-16 14:58:27 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='gelu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, add_prev_output_tokens=False, all_gather_list_size=16384, arch='roberta_small', attention_dropout=0.1, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', classification_head_name='sentence_classification_head', clip_norm=0.0, cpu=False, criterion='sentence_prediction', curriculum=0, data='../../data/fairseq/', data_buffer_size=10, dataset_impl='raw', ddp_backend='c10d', device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=0, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', dropout=0.1, empty_cache_freq=0, encoder_attention_heads=8, encoder_embed_dim=128, encoder_ffn_embed_d

In [55]:
! fairseq-train --help

usage: fairseq-train [-h] [--no-progress-bar] [--log-interval LOG_INTERVAL]
                     [--log-format {json,none,simple,tqdm}]
                     [--tensorboard-logdir TENSORBOARD_LOGDIR] [--seed SEED]
                     [--cpu] [--tpu] [--bf16] [--memory-efficient-bf16]
                     [--fp16] [--memory-efficient-fp16]
                     [--fp16-no-flatten-grads]
                     [--fp16-init-scale FP16_INIT_SCALE]
                     [--fp16-scale-window FP16_SCALE_WINDOW]
                     [--fp16-scale-tolerance FP16_SCALE_TOLERANCE]
                     [--min-loss-scale MIN_LOSS_SCALE]
                     [--threshold-loss-scale THRESHOLD_LOSS_SCALE]
                     [--user-dir USER_DIR]
                     [--empty-cache-freq EMPTY_CACHE_FREQ]
                     [--all-gather-list-size ALL_GATHER_LIST_SIZE]
                     [--model-parallel-size MODEL_PARALLEL_SIZE]
                     [--checkpoint-suffix CHECKPOINT_SUFFIX]
          